In [1]:
# Loading all Packages
# General Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import plotly.express as px
import scipy as spy
import seaborn as sns
import random
from unicodedata import category

# ML Packages
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
import pickle

In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder


model = pickle.load(open("app/src/one_hot_enc_model.sav","rb"))
enc = pickle.load(open("app/src/one_hot_enc.sav","rb"))

model_label = pickle.load(open("app/src/label_model.sav","rb"))
enc_label = pickle.load(open("app/src/label_enc.sav","rb"))

def list_to_pandas(inputs: list) -> pd.DataFrame:

    temp = {"Area": [inputs[0]],"Room": [inputs[1]],"Region": [inputs[2]]}
    temp = pd.DataFrame(temp)

    return temp

# creating  Label Encoding function enabling to properly encode possible future data points for predictions
def OH_encoding(transformData: pd.DataFrame, OH_Encoder: OneHotEncoder, encoderColumn: str) -> pd.DataFrame:

    if "index" in transformData.columns:
        transformData = transformData.drop(["index"], axis=1)
        transformData = transformData.reset_index()
    
    if "index" not in transformData.columns:
        transformData = transformData.reset_index()
    
    temp_frame = pd.DataFrame((OH_Encoder.transform(transformData[[encoderColumn]])).toarray())
    transformData = pd.concat([transformData,temp_frame], axis=1).drop([encoderColumn], axis=1)
    transformData = transformData.drop(["index"], axis=1)
    transformData.columns = transformData.columns.astype(str)
       
    return transformData

def OH_prediction(predict_input: pd.DataFrame) -> int:

    prediction = int(model.predict(predict_input)[0])

    return prediction

In [3]:
test = list_to_pandas([50,3,1019])

In [4]:
test

,Area,Room,Region
0,50,3,1019


In [5]:
encode = OH_encoding(test,enc,"Region")

In [6]:
encode

,Area,Room,0,1,2,3,4,5,6,7,...,59,60,61,62,63,64,65,66,67,68
0,50,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
OH_prediction(encode)

14078776

In [8]:

# creating  Label Encoding function enabling to properly encode possible future data points for label predictions
def label_encoding(transformData: pd.DataFrame, NewColumnName: str, encodePattern: pd.DataFrame, encoderColumn: str) -> pd.DataFrame:
    transformData["temp_col1"] = [int(encodePattern.query("Region == @i")[encoderColumn]) for i in transformData["Region"]]
    transformData = transformData.drop([NewColumnName,"Region"], axis=1).rename(columns={"temp_col1" : NewColumnName})
    return transformData


def label_prediction(predict_input: pd.DataFrame) -> int:

    prediction = int(model_label.predict(predict_input)[0])

    return prediction

In [9]:
test = list_to_pandas([50,3,1019])

In [ ]:
encode = OH_encoding(test,"price_cat_encoded",enc_label,"most_frequent_cat")